In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.special import factorial
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')

In [2]:
df_with_waits = pd.read_csv('surgery_data_with_referral_dates.csv')
df_with_waits.head()

,Activity Type,Provider,Local Patient Identifier,LSOA,HRG,HRG Description,Fin Year,Fin Month,Activity Date,Independent Sector,...,Waiting_Time_Days,Waiting_Time_Weeks,Urgency_Level,Breach_18_Weeks,Breach_52_Weeks,Referral_Year,Referral_Month,Referral_Quarter,Referral_Day_of_Week,Provider_Type
0,Inpatient/Day Case,CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION ...,270506,E01018242,BZ31B,"Very Major, Cataract or Lens Procedures, with ...",2022/23,6,2022-09-12,No,...,168.962761,24.137537,3,True,False,2022,3,1,Monday,NHS Trust
1,Inpatient/Day Case,CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION ...,1319077,E01018159,BZ31B,"Very Major, Cataract or Lens Procedures, with ...",2022/23,9,2022-12-19,No,...,170.113204,24.301886,2,True,False,2022,7,3,Saturday,NHS Trust
2,Inpatient/Day Case,NORTH WEST ANGLIA NHS FOUNDATION TRUST,DIS3078757,E01018192,BZ31B,"Very Major, Cataract or Lens Procedures, with ...",2022/23,12,2023-03-28,No,...,220.916901,31.559557,3,True,False,2022,8,3,Saturday,NHS Trust
3,Inpatient/Day Case,NORTH WEST ANGLIA NHS FOUNDATION TRUST,DIS3027712,E01015599,BZ31B,"Very Major, Cataract or Lens Procedures, with ...",2022/23,11,2023-02-21,No,...,94.878105,13.554015,1,False,False,2022,11,4,Saturday,NHS Trust
4,Inpatient/Day Case,CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION ...,930216,E01018051,BZ31B,"Very Major, Cataract or Lens Procedures, with ...",2022/23,9,2022-12-06,No,...,234.570060,33.510009,3,True,False,2022,4,2,Saturday,NHS Trust


In [3]:
class MMcQueue:
    """
    M/M/c Queue Model for Healthcare Systems
    
    M/M/c represents:
    - M: Markovian (exponential) arrival times
    - M: Markovian (exponential) service times  
    - c: Number of servers (surgeons/theatres)
    """
    
    def __init__(self, arrival_rate, service_rate, num_servers, name="Queue"):
        """
        Initialize M/M/c queue
        
        Parameters:
        - arrival_rate (λ): Average arrivals per time unit
        - service_rate (μ): Average service rate per server per time unit
        - num_servers (c): Number of servers
        - name: Queue identifier
        """
        self.name = name
        self.lambda_ = arrival_rate
        self.mu = service_rate
        self.c = num_servers
        self.rho = arrival_rate / (num_servers * service_rate) if num_servers > 0 and service_rate > 0 else float('inf')
        
    def is_stable(self):
        """Check if queue is stable (ρ < 1)"""
        return self.rho < 1
    
    def calculate_p0(self):
        """Calculate probability of zero patients in system"""
        if not self.is_stable():
            return 0
        
        # Sum for n = 0 to c-1
        sum_term = sum([(self.lambda_/self.mu)**n / factorial(n) for n in range(self.c)])
        
        # Term for n = c to infinity
        last_term = ((self.lambda_/self.mu)**self.c / factorial(self.c)) * (1/(1-self.rho))
        
        p0 = 1 / (sum_term + last_term)
        return p0
    
    def calculate_metrics(self):
        """Calculate all queue performance metrics"""
        if not self.is_stable():
            return {
                'name': self.name,
                'stable': False,
                'utilization': self.rho,
                'servers': self.c,
                'arrival_rate': self.lambda_,
                'service_rate': self.mu,
                'avg_queue_length': float('inf'),
                'avg_system_length': float('inf'),
                'avg_waiting_time': float('inf'),
                'avg_system_time': float('inf'),
                'probability_wait': 1.0,
                'probability_immediate_service': 0.0,
                'avg_waiting_time_days': float('inf'),
                'avg_system_time_days': float('inf')
            }
        
        p0 = self.calculate_p0()
        
        # Erlang C formula - probability that arriving customer must wait
        pc = ((self.lambda_/self.mu)**self.c / factorial(self.c)) * p0 / (1 - self.rho)
        
        # Average number in queue (Lq)
        lq = pc * self.rho / (1 - self.rho)
        
        # Average number in system (L) - Little's Law
        l = lq + self.lambda_/self.mu
        
        # Average waiting time in queue (Wq)
        wq = lq / self.lambda_
        
        # Average time in system (W)
        w = l / self.lambda_
        
        # Probability of immediate service
        p_immediate = 1 - pc
        
        return {
            'name': self.name,
            'stable': True,
            'utilization': self.rho,
            'servers': self.c,
            'arrival_rate': self.lambda_,
            'service_rate': self.mu,
            'avg_queue_length': lq,
            'avg_system_length': l,
            'avg_waiting_time_hours': wq,
            'avg_waiting_time_days': wq * 24 if wq < float('inf') else float('inf'),  # Convert to days
            'avg_system_time_hours': w,
            'avg_system_time_days': w * 24 if w < float('inf') else float('inf'),  # Convert to days
            'probability_wait': pc,
            'probability_immediate_service': p_immediate,
            'p0': p0
        }

In [4]:
class HealthcareQueueAnalyzerWithBacklog:
    """
    Enhanced analyzer for healthcare queueing systems
    Accounts for existing waiting lists and true demand
    """
    
    def __init__(self, df_with_waits):
        """
        Initialize with the dataframe containing waiting times
        """
        self.df = df_with_waits.copy()
        self.providers = self.df['Provider'].unique()
        self.procedures = self.df['HRG Description'].unique()
        
    def calculate_true_demand_rates(self):
        """
        Calculate TRUE demand (not just completed procedures)
        Accounts for both completed procedures AND waiting backlog
        """
        # Ensure datetime columns
        df_temp = self.df.copy()
        df_temp['Activity Date'] = pd.to_datetime(df_temp['Activity Date'])
        df_temp['Referral_Date'] = pd.to_datetime(df_temp['Referral_Date'])
        
        # Add year-month columns
        df_temp['Activity_Year_Month'] = df_temp['Activity Date'].dt.to_period('M')
        df_temp['Referral_Year_Month'] = df_temp['Referral_Date'].dt.to_period('M')
        
        # Calculate average REFERRALS per month (true demand)
        monthly_referrals = df_temp.groupby(['Provider', 'Referral_Year_Month']).size()
        avg_monthly_referrals = monthly_referrals.groupby('Provider').mean()
        
        # Calculate current waiting list size per provider using Little's Law
        waiting_list = {}
        
        for provider in self.providers:
            provider_data = df_temp[df_temp['Provider'] == provider]
            if len(provider_data) > 0:
                # Average waiting time for this provider
                avg_wait_days = provider_data['Waiting_Time_Days'].mean()
                # Daily arrival rate
                if provider in avg_monthly_referrals:
                    daily_arrival_rate = avg_monthly_referrals[provider] / 30
                    # Little's Law: L = λW
                    waiting_list[provider] = daily_arrival_rate * avg_wait_days
                else:
                    waiting_list[provider] = 0
            else:
                waiting_list[provider] = 0
        
        # Convert to hourly demand rates
        hourly_demand_rates = avg_monthly_referrals / (22 * 8)  # 22 working days, 8 hours per day
        
        return hourly_demand_rates.to_dict(), waiting_list
    
    def estimate_service_rates(self):
        """
        Estimate service rates based on procedure complexity using Cambridgeshire & Peterborough data
        """
        # Operating times from the PDF (in minutes)
        hrg_operating_times = {
            'Minor, Cataract or Lens Procedures': 7.5,
            'Intermediate, Cataract or Lens Procedures, with CC Score 0-1': 12.5,
            'Intermediate, Cataract or Lens Procedures, with CC Score 2+': 17.5,
            'Phacoemulsification Cataract Extraction and Lens Implant, with CC Score 0-1': 10,
            'Phacoemulsification Cataract Extraction and Lens Implant, with CC Score 2-3': 12.5,
            'Phacoemulsification Cataract Extraction and Lens Implant, with CC Score 4+': 17.5,
            'Complex, Cataract or Lens Procedures, with CC Score 0-1': 25,
            'Complex, Cataract or Lens Procedures, with CC Score 2+': 25,
            'Very Major, Cataract or Lens Procedures, with CC Score 0-1': 35,
            'Very Major, Cataract or Lens Procedures, with CC Score 2+': 50
        }
        
        # Prep and turnover times
        prep_and_turnover_by_procedure = {
            'Minor': 5,
            'Intermediate': 10,
            'Phacoemulsification': 10,
            'Complex': 15,
            'Very Major': 20
        }
        
        provider_service_rates = {}
        
        for provider in self.providers:
            provider_data = self.df[self.df['Provider'] == provider]
            
            if len(provider_data) > 0:
                total_weighted_time = 0
                total_cases = 0
                
                # Calculate weighted average time based on case mix
                hrg_distribution = provider_data['HRG Description'].value_counts()
                
                for hrg_desc, count in hrg_distribution.items():
                    op_time = hrg_operating_times.get(hrg_desc, 20)
                    
                    # Determine prep time
                    if 'Minor' in hrg_desc:
                        prep_time = prep_and_turnover_by_procedure['Minor']
                    elif 'Intermediate' in hrg_desc:
                        prep_time = prep_and_turnover_by_procedure['Intermediate']
                    elif 'Phacoemulsification' in hrg_desc:
                        prep_time = prep_and_turnover_by_procedure['Phacoemulsification']
                    elif 'Complex' in hrg_desc:
                        prep_time = prep_and_turnover_by_procedure['Complex']
                    elif 'Very Major' in hrg_desc:
                        prep_time = prep_and_turnover_by_procedure['Very Major']
                    else:
                        prep_time = 10
                    
                    total_theatre_time = op_time + prep_time
                    total_weighted_time += total_theatre_time * count
                    total_cases += count
                
                avg_theatre_time_minutes = total_weighted_time / total_cases if total_cases > 0 else 30
                
                # Adjust for provider type
                if 'CAMBRIDGE UNIVERSITY HOSPITALS' in provider or 'TEACHING' in provider:
                    avg_theatre_time_minutes *= 1.3  # Teaching hospitals slower
                elif 'SPAMEDICA' in provider:
                    avg_theatre_time_minutes *= 0.85  # Specialized providers faster
                elif 'ANGLIA COMMUNITY EYE SERVICE' in provider or 'ACES' in provider:
                    avg_theatre_time_minutes *= 0.9
                elif 'NEWMEDICA' in provider or 'OPTEGRA' in provider:
                    avg_theatre_time_minutes *= 0.9
                elif 'NHS FOUNDATION TRUST' in provider:
                    avg_theatre_time_minutes *= 1.0
                else:
                    avg_theatre_time_minutes *= 0.95
                
                # Convert to procedures per hour
                avg_theatre_time_hours = avg_theatre_time_minutes / 60
                provider_service_rates[provider] = 1 / avg_theatre_time_hours
            else:
                provider_service_rates[provider] = 2.0  # Default 2 procedures per hour
        
        return provider_service_rates
    
    def calculate_actual_capacity_constraints(self):
        """
        Calculate actual capacity based on historical throughput
        """
        df_temp = self.df.copy()
        df_temp['Activity Date'] = pd.to_datetime(df_temp['Activity Date'])
        df_temp['Year_Month'] = df_temp['Activity Date'].dt.to_period('M')
        
        # Actual procedures completed per month
        monthly_completed = df_temp.groupby(['Provider', 'Year_Month']).size()
        
        # Maximum and average monthly capacity demonstrated
        max_monthly_capacity = monthly_completed.groupby('Provider').max()
        avg_monthly_capacity = monthly_completed.groupby('Provider').mean()
        
        # Estimate number of theatres based on demonstrated capacity
        service_rates = self.estimate_service_rates()
        estimated_servers = {}
        
        for provider in self.providers:
            if provider in max_monthly_capacity and provider in service_rates:
                max_procedures = max_monthly_capacity[provider]
                procedures_per_hour_per_theatre = service_rates[provider]
                total_hours_needed = max_procedures / procedures_per_hour_per_theatre
                # Number of theatres (22 days, 8 hours per day)
                num_theatres = total_hours_needed / (22 * 8)
                estimated_servers[provider] = max(1, int(np.ceil(num_theatres)))
            else:
                estimated_servers[provider] = 1
        
        return avg_monthly_capacity.to_dict(), estimated_servers
    
    def analyze_systems_with_backlog(self):
        """
        Analyze both fragmented and consolidated systems considering backlogs
        """
        # Get true demand and waiting lists
        demand_rates, waiting_lists = self.calculate_true_demand_rates()
        
        # Get actual capacity constraints
        capacity_rates, num_servers = self.calculate_actual_capacity_constraints()
        
        # Service rates
        service_rates = self.estimate_service_rates()
        
        # Create fragmented system analysis
        fragmented_metrics = []
        
        for provider in self.providers:
            if provider in demand_rates and demand_rates[provider] > 0:
                arrival_rate = demand_rates[provider]
                service_rate = service_rates.get(provider, 2.0)
                servers = num_servers.get(provider, 1)
                
                queue = MMcQueue(
                    arrival_rate=arrival_rate,
                    service_rate=service_rate,
                    num_servers=servers,
                    name=provider[:40] + "..." if len(provider) > 40 else provider
                )
                
                metrics = queue.calculate_metrics()
                metrics['provider'] = provider
                metrics['current_backlog'] = waiting_lists.get(provider, 0)
                metrics['monthly_referrals'] = arrival_rate * 22 * 8  # Convert back to monthly
                metrics['monthly_capacity'] = capacity_rates.get(provider, 0)
                
                # Get actual waiting times from data
                provider_data = self.df[self.df['Provider'] == provider]
                if len(provider_data) > 0:
                    metrics['avg_waiting_days_actual'] = provider_data['Waiting_Time_Days'].mean()
                    metrics['median_waiting_days_actual'] = provider_data['Waiting_Time_Days'].median()
                    metrics['p90_waiting_days_actual'] = provider_data['Waiting_Time_Days'].quantile(0.9)
                else:
                    metrics['avg_waiting_days_actual'] = 0
                    metrics['median_waiting_days_actual'] = 0
                    metrics['p90_waiting_days_actual'] = 0
                
                fragmented_metrics.append(metrics)
        
        fragmented_df = pd.DataFrame(fragmented_metrics)
        
        # Create consolidated system
        total_demand = sum(demand_rates.values())
        total_servers = sum(num_servers.values())
        
        # Weighted average service rate
        if total_demand > 0:
            weights = {p: demand_rates.get(p, 0)/total_demand for p in self.providers}
            avg_service_rate = sum(service_rates.get(p, 2.0) * weights.get(p, 0) for p in self.providers)
        else:
            avg_service_rate = 2.0
        
        consolidated_queue = MMcQueue(
            arrival_rate=total_demand,
            service_rate=avg_service_rate,
            num_servers=total_servers,
            name="Consolidated PTL System"
        )
        
        consolidated_metrics = consolidated_queue.calculate_metrics()
        consolidated_metrics['total_current_backlog'] = sum(waiting_lists.values())
        consolidated_metrics['total_monthly_referrals'] = total_demand * 22 * 8
        
        return fragmented_df, consolidated_metrics

In [5]:
# Initialize enhanced analyzer
print("Initializing Enhanced Healthcare Queue Analyzer with Backlog Consideration...")
analyzer = HealthcareQueueAnalyzerWithBacklog(df_with_waits)

# Run the analysis
print("\nAnalyzing healthcare systems with existing waiting lists...")
fragmented_df, consolidated_metrics = analyzer.analyze_systems_with_backlog()

# Print detailed results
print("\n" + "="*80)
print("ENHANCED FRAGMENTED SYSTEM ANALYSIS (Current State)")
print("="*80)

# Separate stable and unstable queues
stable_queues = fragmented_df[fragmented_df['stable']]
unstable_queues = fragmented_df[~fragmented_df['stable']]

print(f"\nQueue Stability:")
print(f"- Stable queues: {len(stable_queues)} out of {len(fragmented_df)}")
print(f"- Unstable queues: {len(unstable_queues)} (demand exceeds capacity)")

if len(unstable_queues) > 0:
    print("\nUnstable Providers (need immediate attention):")
    for _, provider in unstable_queues.iterrows():
        print(f"- {provider['name']}: ρ = {provider['utilization']:.2f}")

if len(stable_queues) > 0:
    print(f"\nFragmented System Metrics (stable queues only):")
    print(f"- Average utilization: {stable_queues['utilization'].mean():.1%}")
    print(f"- Average current backlog: {stable_queues['current_backlog'].mean():.0f} patients")
    print(f"- Total patients in backlog: {stable_queues['current_backlog'].sum():.0f}")
    print(f"- Average actual waiting time: {stable_queues['avg_waiting_days_actual'].mean():.1f} days")
    print(f"- Average theoretical queue length: {stable_queues['avg_queue_length'].mean():.1f} patients")

print("\n" + "="*80)
print("CONSOLIDATED SYSTEM ANALYSIS (Proposed State)")
print("="*80)

print(f"\nConsolidated System Metrics:")
print(f"- System stable: {consolidated_metrics['stable']}")
print(f"- Total servers (theatres): {consolidated_metrics['servers']}")
print(f"- System utilization: {consolidated_metrics['utilization']:.1%}")
print(f"- Total current backlog: {consolidated_metrics['total_current_backlog']:.0f} patients")
print(f"- Theoretical avg queue length: {consolidated_metrics['avg_queue_length']:.1f} patients")
print(f"- Theoretical avg waiting time: {consolidated_metrics['avg_waiting_time_days']:.1f} days")
print(f"- Probability of waiting: {consolidated_metrics['probability_wait']:.1%}")

Initializing Enhanced Healthcare Queue Analyzer with Backlog Consideration...

Analyzing healthcare systems with existing waiting lists...

ENHANCED FRAGMENTED SYSTEM ANALYSIS (Current State)

Queue Stability:
- Stable queues: 81 out of 81
- Unstable queues: 0 (demand exceeds capacity)

Fragmented System Metrics (stable queues only):
- Average utilization: 2.3%
- Average current backlog: 42 patients
- Total patients in backlog: 3426
- Average actual waiting time: 104.4 days
- Average theoretical queue length: 0.0 patients

CONSOLIDATED SYSTEM ANALYSIS (Proposed State)

Consolidated System Metrics:
- System stable: True
- Total servers (theatres): 82
- System utilization: 2.5%
- Total current backlog: 3426 patients
- Theoretical avg queue length: 0.0 patients
- Theoretical avg waiting time: 0.0 days
- Probability of waiting: 0.0%


In [6]:
# Calculate improvements
print("\n" + "="*80)
print("IMPROVEMENT ANALYSIS")
print("="*80)

if len(stable_queues) > 0:
    # Calculate weighted averages for fragmented system
    total_backlog = stable_queues['current_backlog'].sum()
    weights = stable_queues['current_backlog'] / total_backlog if total_backlog > 0 else stable_queues['current_backlog'] * 0
    
    fragmented_avg_wait_actual = (stable_queues['avg_waiting_days_actual'] * weights).sum()
    fragmented_avg_utilization = stable_queues['utilization'].mean()
    
    # Key improvements
    print(f"\n1. WAITING TIME REDUCTION:")
    print(f"   Current average (actual): {fragmented_avg_wait_actual:.1f} days")
    print(f"   Consolidated (theoretical): {consolidated_metrics['avg_waiting_time_days']:.1f} days")
    if fragmented_avg_wait_actual > 0:
        wait_reduction = (fragmented_avg_wait_actual - consolidated_metrics['avg_waiting_time_days']) / fragmented_avg_wait_actual * 100
        print(f"   Potential reduction: {wait_reduction:.1f}%")
    
    print(f"\n2. UTILIZATION IMPROVEMENT:")
    print(f"   Current (fragmented): {fragmented_avg_utilization:.1%}")
    print(f"   Consolidated: {consolidated_metrics['utilization']:.1%}")
    
    print(f"\n3. EQUITY IMPROVEMENT:")
    wait_variance = stable_queues['avg_waiting_days_actual'].var()
    print(f"   Current variance in waiting times: {wait_variance:.0f} days²")
    print(f"   Consolidated: 0 days² (all patients wait equally)")
    
    print(f"\n4. CAPACITY ISSUES:")
    print(f"   Unstable queues in current system: {len(unstable_queues)}")
    print(f"   Unstable queues in consolidated: {'0 (pooled capacity handles demand)' if consolidated_metrics['stable'] else '1 (total demand exceeds total capacity)'}")


IMPROVEMENT ANALYSIS

1. WAITING TIME REDUCTION:
   Current average (actual): 105.6 days
   Consolidated (theoretical): 0.0 days
   Potential reduction: 100.0%

2. UTILIZATION IMPROVEMENT:
   Current (fragmented): 2.3%
   Consolidated: 2.5%

3. EQUITY IMPROVEMENT:
   Current variance in waiting times: 1308 days²
   Consolidated: 0 days² (all patients wait equally)

4. CAPACITY ISSUES:
   Unstable queues in current system: 0
   Unstable queues in consolidated: 0 (pooled capacity handles demand)
